In [ ]:
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
import re
import datetime

app = Flask(__name__)
CORS(app)
UNIVERSITY_PROGRAMS = {
    'computer science': {
        'requirements': 'High school diploma, Math and Physics recommended, minimum GPA 3.0, English proficiency.',
        'deadline': '2026-01-15',
        'duration': '4 years (undergraduate)'
    },
    'business administration': {
        'requirements': 'High school diploma, personal statement, 2 recommendation letters.',
        'deadline': '2026-02-01',
        'duration': '4 years (undergraduate)'
    }
}

MEDICAL_DEPARTMENTS = {
    'cardiology': {'doctors': ['Dr. Ayesha Khan', 'Dr. Bilal Ahmed'], 'phone': '+92-21-555-1000'},
    'dermatology': {'doctors': ['Dr. Sana Malik'], 'phone': '+92-21-555-1010'}
}

HOSPITAL_INFO = {
    'emergency': 'Open 24/7. Emergency number: +92-21-999',
    'visiting_hours': 'Daily 4:00 PM - 8:00 PM',
    'rooms': {'icu': 2, 'ward': 10, 'private': 5}
}

HOTEL_ROOMS = {
    'standard': {'price': 50, 'amenities': ['Free WiFi', 'TV', 'AC']},
    'deluxe': {'price': 90, 'amenities': ['All standard + Minibar', 'City view']},
    'suite': {'price': 150, 'amenities': ['Living area', 'Complimentary breakfast']}
}

RESTAURANT_MENU = {
    'starters': ['Garlic Bread', 'Chicken Wings'],
    'mains': ['Grilled Chicken', 'Pasta Alfredo'],
    'desserts': ['Chocolate Cake']
}

LIBRARY_BOOKS = [
    {'title': 'Introduction to Algorithms', 'author': 'Cormen', 'available': True, 'due_date': None},
    {'title': 'Database Systems', 'author': 'Elmasri', 'available': False, 'due_date': '2025-12-20'}
]
def find_program(query):
    q = query.lower()
    for name in UNIVERSITY_PROGRAMS:
        if name in q:
            return name, UNIVERSITY_PROGRAMS[name]
    return None, None
def find_department(query):
    q = query.lower()
    for dept in MEDICAL_DEPARTMENTS:
        if dept in q:
            return dept, MEDICAL_DEPARTMENTS[dept]
    return None, None
def get_room_availability():
    return HOSPITAL_INFO['rooms']
def find_hotel_room(query):
    q = query.lower()
    for r in HOTEL_ROOMS:
        if r in q:
            return r, HOTEL_ROOMS[r]
    return None, None
def find_book(query):
    q = query.lower()
    for b in LIBRARY_BOOKS:
        if b['title'].lower().startswith(q) or q in b['title'].lower() or q in b['author'].lower():
            return b
    return None
def university_bot(msg):
    if re.search(r'deadline|apply|application', msg, re.I):
        name, p = find_program(msg)
        if p:
            return f"Application deadline for {name.title()} is {p['deadline']}. Requirements: {p['requirements']}"
        return "General application deadlines: Fall intake deadline is 2026-01-15. Tell me which program you are interested in."
    if re.search(r'requirement|prerequisite', msg, re.I):
        name, p = find_program(msg)
        if p:
            return f"Requirements for {name.title()}: {p['requirements']}"
        return "Please specify a program (e.g., Computer Science) to get detailed requirements."
    if re.search(r'program|duration|how long', msg, re.I):
        name, p = find_program(msg)
        if p:
            return f"{name.title()} program: {p['duration']} (Deadline: {p['deadline']})"
        return "We offer various programs like Computer Science and Business Administration. Which one interests you?"
    return "I can help with admission deadlines, requirements, and program info. Ask about a specific program like 'Computer Science'."
def medical_center_bot(msg):
    if re.search(r'appointment|book|schedule', msg, re.I):
        dept, info = find_department(msg)
        if dept:
            return f"To book an appointment with {dept.title()}, call {info['phone']} or visit our appointments page. Available doctors: {', '.join(info['doctors'])}."
        return "To book an appointment, tell me the department (e.g., Cardiology) or call +92-21-555-1000."
    if re.search(r'doctor|physician|dr\.', msg, re.I):
        dept, info = find_department(msg)
        if dept:
            return f"Doctors in {dept.title()}: {', '.join(info['doctors'])}."
        return "Which department's doctors are you looking for?"
    return "I can help with appointments, departments, and doctors. Which department do you need?"
def hospital_bot(msg):
    if re.search(r'emergency|ambulance', msg, re.I):
        return HOSPITAL_INFO['emergency']
    if re.search(r'visiting|visit', msg, re.I):
        return HOSPITAL_INFO['visiting_hours']
    if re.search(r'room|availability|icu|ward|private', msg, re.I):
        rooms = get_room_availability()
        return f"Room availability — ICU: {rooms['icu']}, Ward: {rooms['ward']}, Private: {rooms['private']}. Contact admissions to reserve."
    return "I can provide emergency info, visiting hours, and room availability. What would you like to know?"
def hotel_bot(msg):
    if re.search(r'room types|rooms|type', msg, re.I):
        types = ', '.join([f"{k.title()} (${v['price']}/night)" for k, v in HOTEL_ROOMS.items()])
        return f"We offer: {types}. Which would you like to book?"
    if re.search(r'price|cost|rate', msg, re.I):
        r, info = find_hotel_room(msg)
        if info:
            return f"{r.title()} room: ${info['price']} per night. Amenities: {', '.join(info['amenities'])}."
        return "Standard room starts at $50/night. Tell me which room type you're interested in."
    if re.search(r'book|reservation', msg, re.I):
        return "To make a booking, please provide your check-in and check-out dates and number of guests, or call +92-21-555-2000."
    return "I can help with room types, prices, amenities, and bookings. What do you need?"
def restaurant_bot(msg):
    if re.search(r'menu|what do you have|food', msg, re.I):
        parts = '\n'.join([f"{k.title()}: {', '.join(v)}" for k, v in RESTAURANT_MENU.items()])
        return f"Here's our menu:\n{parts}"
    if re.search(r'reservation|table|reserve', msg, re.I):
        return "Reservations: Please tell me date, time, and party size, or call +92-21-555-3000."
    if re.search(r'order|track', msg, re.I):
        return "To track an order, please provide your order ID."
    return "I can show the menu, help with reservations, or track orders. What would you like?"
def library_bot(msg):
    if re.search(r'find|search|book', msg, re.I):
        b = find_book(msg)
        if b:
            avail = 'Available' if b['available'] else f"Checked out (due {b['due_date']})"
            return f"{b['title']} by {b['author']} — {avail}"
        return "No matching book found. Try the exact title or author."
    if re.search(r'due date|due|return', msg, re.I):
        borrowed = [b for b in LIBRARY_BOOKS if not b['available']]
        if borrowed:
            lines = '; '.join([f"{b['title']} due {b['due_date']}" for b in borrowed])
            return f"Borrowed books: {lines}"
        return "You have no books due."
    if re.search(r'opening|hours|when open', msg, re.I):
        return "Library hours: Mon-Fri 8:00 AM - 8:00 PM; Sat 9:00 AM - 5:00 PM; Sun closed."
    return "I can help find books, tell due dates, and opening hours. What do you need?"


def creative_bot(msg):
    if re.search(r'tour|visit|campus', msg, re.I):
        return "Campus tour: Guided tours run Mon-Fri at 10:00 and 14:00. Meet at the main gate. Would you like to register?"
    if re.search(r'register|sign up', msg, re.I):
        return "You can register for a campus tour by providing your name and preferred date."
    return "I'm the Campus Tour Assistant. I can help you schedule tours and learn about campus highlights."
BOT_MAP = {
    'university': university_bot,
    'medical_center': medical_center_bot,
    'hospital': hospital_bot,
    'hotel': hotel_bot,
    'restaurant': restaurant_bot,
    'library': library_bot,
    'creative': creative_bot
}
# Web UI (single-page)
INDEX_HTML = '''
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Multi Bot Demo</title>
  <style>
    body { font-family: Arial, sans-serif; background:#f2f4f7; padding:20px }
    .container { max-width:900px; margin:0 auto; background:#fff; padding:20px; border-radius:8px; box-shadow:0 6px 18px rgba(0,0,0,0.06)}
    .header { display:flex; gap:10px; align-items:center }
    .chat { border:1px solid #e1e4e8; height:400px; overflow:auto; padding:10px; border-radius:6px; background:#fafafa }
    .message { margin:8px 0; }
    .user { text-align:right }
    .bot { text-align:left }
    .bubble { display:inline-block; padding:10px 12px; border-radius:12px; max-width:75% }
    .user .bubble { background:#2b8be6; color:white }
    .bot .bubble { background:#e9eef8; color:#111 }
    .controls { margin-top:10px; display:flex; gap:8px }
    select, input[type=text] { padding:8px; border-radius:6px; border:1px solid #ccd; flex:1 }
    button { padding:8px 12px; border-radius:6px; border:none; background:#2b8be6; color:white }
    footer { margin-top:12px; color:#666; font-size:13px }
  </style>
</head>
<body>
  <div class="container">
    <div class="header">
      <h2>Multi Chatbot Demo</h2>
      <select id="botSelect">
        <option value="university">University Admission Bot</option>
        <option value="medical_center">Medical Center Bot</option>
        <option value="hospital">Hospital Bot</option>
        <option value="hotel">Hotel Bot</option>
        <option value="restaurant">Restaurant Bot</option>
        <option value="library">Library Assistant</option>
        <option value="creative">Campus Tour (Creative)</option>
      </select>
    </div>

    <div id="chat" class="chat"></div>

    <div class="controls">
      <input id="messageInput" type="text" placeholder="Type your message... (e.g., 'What are admission deadlines for Computer Science?')" />
      <button id="sendBtn">Send</button>
    </div>
    <footer>Tip: Try asking about deadlines, appointments, room availability, or menus.</footer>
  </div>

  <script>
    const chatEl = document.getElementById('chat')
    const input = document.getElementById('messageInput')
    const sendBtn = document.getElementById('sendBtn')
    const botSelect = document.getElementById('botSelect')

    function appendMessage(text, who='bot'){
      const div = document.createElement('div')
      div.className = 'message ' + (who === 'user' ? 'user' : 'bot')
      const span = document.createElement('span')
      span.className = 'bubble'
      span.textContent = text
      div.appendChild(span)
      chatEl.appendChild(div)
      chatEl.scrollTop = chatEl.scrollHeight
    }

    async function sendMessage(){
      const msg = input.value.trim()
      if(!msg) return
      appendMessage(msg, 'user')
      input.value = ''
      const bot = botSelect.value
      try{
        const res = await fetch('/api/chat', {
          method: 'POST', headers: {'Content-Type': 'application/json'},
          body: JSON.stringify({bot, message: msg})
        })
        const data = await res.json()
        appendMessage(data.reply || 'Sorry, no answer available.')
      } catch(err){
        appendMessage('Error contacting server: ' + err.message)
      }
    }

    sendBtn.addEventListener('click', sendMessage)
    input.addEventListener('keydown', (e)=>{ if(e.key === 'Enter') sendMessage() })

    // Greet
    appendMessage('Hello! Select a bot and ask me something. For example: "What are the library opening hours?"')
  </script>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(INDEX_HTML)
@app.route('/api/chat', methods=['POST'])
def api_chat():
    data = request.get_json() or {}
    bot = data.get('bot')
    message = data.get('message', '')
    if not bot or bot not in BOT_MAP:
        return jsonify({'error': 'Invalid bot selected', 'reply': 'Please select a valid bot.'}), 400
    if not message.strip():
        return jsonify({'error': 'Empty message', 'reply': 'Please type a message.'}), 400
    try:
        reply = BOT_MAP[bot](message)
    except Exception as e:
        reply = 'Sorry, something went wrong when processing your message.'
    return jsonify({'reply': reply, 'bot': bot, 'timestamp': datetime.datetime.utcnow().isoformat() + 'Z'})
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
